Read-Only and Computed Properties
Although write-only properties are not that common, read-only properties (i.e. that define a getter but not a setter) are quite common for a number of things.

Of course, we can create read-only properties, but since nothing is private, at best we are "suggesting" to the users of our class they should treat the property as read-only. There's always a way to hack around that of course.

But still, it's good to be able to at least explicitly indicate to a user that a property is meant to be read-only.

The use case I'm going to focus on in this video, is one of computed properties. Those are properties that may not actually have a backing variable, but are instead calculated on the fly.

Consider this simple example of a Circle class where we can read/write the radius of the circle, but want a computed property for the area. We don't need to store the area value, we can alway calculate it given the current radius value.

In [1]:
from math import pi

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @property
    def area(self):
        print('calculating area...')
        return pi * (self.radius ** 2)

In [2]:
c =Circle(1)
c.area

calculating area...


3.141592653589793

We could certainly just use a class method area(), but the area is more a property of the circle, so it makes more sense to just retrive it as a property, without the extra () to make the call.

The advantage of how we did this is that shoudl the radius of the circle ever change, the area property will immediately reflect that.

On the other hand, it's also a weakness - every time we need the area of the circle, it gets recalculated, even if the radius has not changed!

In [3]:
c = Circle(2)

In [4]:
c.area
c.area

calculating area...
calculating area...


12.566370614359172

So now we can use properties to fix this problem without breaking our interface!

We are going to cache the area value, and only-recalculate it if the radius has changed.

In order for us to know if the radius has changed, we are going to make it into a property, and the setter will keep track of whether the radius is set, in which case it will invalidate the cached area value.

In [5]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
        self._area = None
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, value):
        # if radius value is set we invalidate our cached _area value
        # we could make this more intelligent and see if the radius has actually changed
        # but keeping it simple
        self._area = None
        # we could even add validation here, like value has to be numeric, non-negative, etc
        self._radius = value
        
    @property
    def area(self):
        if self._area is None:
            # value not cached - calculate it
            print('Calculating area...')
            self._area = pi * (self.radius ** 2)
        return self._area

In [6]:
c = Circle(2)


In [7]:
c.area
c.area

Calculating area...


12.566370614359172

here are a lot of other uses for calculate properties.

Some properties may even do a lot work, like retrieving data from a database, making a call to some external API, and 

Let's write a class that takes a URL, downloads the web page for that URL and provides us some metrics on that URL - like how long it took to download, the size (in bytes) of the page.

Although I am going to use the urllib module for this, I strongly recommend you use the requests 3rd party library instead: http://docs.python-requests.org

In [10]:
import urllib
from time import perf_counter

In [12]:
class WebPage:
    def __init__(self, url):
        self.url = url
        self._page = None
        self._load_time_secs = None
        self._page_size = None
        
    @property
    def url(self):
        return self._url
    
    @url.setter
    def url(self, value):
        self._url = value
        self._page = None
        # we'll lazy load the page - i.e. we wait until some property is requested
        
    @property
    def page(self):
        if self._page is None:
            self.download_page()
        return self._page
    
    @property
    def page_size(self):
        if self._page is None:
            # need to first download the page
            self.download_page()
        return self._page_size
        
    @property
    def time_elapsed(self):
        if self._page is None:
            self.download_page()
        return self._load_time_secs
            
    def download_page(self):
        self._page_size = None
        self._load_time_secs = None
        start_time = perf_counter()
        with urllib.request.urlopen(self.url) as f:
            self._page = f.read()
        end_time = perf_counter()
        
        self._page_size = len(self._page)
        self._load_time_secs = end_time - start_time


In [13]:
urls = [
    'https://www.google.com',
    'https://www.python.org',
    'https://www.yahoo.com'
]

for url in urls:
    page = WebPage(url)
    print(f'{url} \tsize={format(page.page_size, "_")} \telapsed={page.time_elapsed:.2f} secs')

https://www.google.com 	size=14_318 	elapsed=0.29 secs
https://www.python.org 	size=49_694 	elapsed=0.38 secs
https://www.yahoo.com 	size=452_621 	elapsed=1.35 secs


### Deleting Properties

Just like we can delete an attribute from an instance object, we can also delete a property from an instance object.

Note that this action simply runs the deleter method, but the propertu remains defined on the class. It does not remove the property from the class, instead it is generally used to remove the property value from the instance.

Properties, like attributes, can be deleted by using the del keyword, or the delattr function.

In [14]:
class Person:
    def __init__(self, name):
        self.name = name

    def get_name(self):
        print('getting name property value...')
        return self._name
    
    def set_name(self, value):
        print(f'setting name property to {value}...')
        self._name = value
    
    def del_name(self):
        # delete the underlying data
        print('deleting name property value...')
        del self._name
        
    name = property(fget=get_name, fset=set_name, fdel=del_name, doc='Person name.')

In [15]:
p = Person('Guido')

setting name property to Guido...


In [16]:
p.name

getting name property value...


'Guido'

In [17]:
p.__dict__


{'_name': 'Guido'}

In [18]:
del p.name

deleting name property value...


In [20]:
# As we can see, the underlying _name attribute is no longer present in the instance dictionary:

p.__dict__

{}

In [21]:
try:
    print(p.name)
except AttributeError as ex:
    print(ex)

getting name property value...
'Person' object has no attribute '_name'



As you can see, the property deletion did not remove the property definition, that still exists.

Alternatively, we can use the delattr function as well:

In [22]:
p = Person('Raymond')

setting name property to Raymond...


In [23]:
delattr(p, 'name')

deleting name property value...


And we can of course use the decorator syntax as well:`m

In [25]:
class Person:
    def __init__(self, name):
        self.name = name

    @property
    def name(self):
        print('getting name property value...')
        return self._name
    
    @name.setter
    def name(self, value):
        """Person name"""
        print(f'setting name property to {value}...')
        self._name = value
    
    @name.deleter
    def name(self):
        # delete the underlying data
        print('deleting name property value...')
        del self._name

In [26]:
p = Person('Alex')

setting name property to Alex...


In [27]:
p.name

getting name property value...


'Alex'

In [28]:
del p.name

deleting name property value...
